In [1]:
#data preprocessing for torch
import torch
import numpy as np
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
import torch.nn as nn
import torchaudio

from dataset.data import MineData
from models import models

In [ ]:
import random
# Load dataset
mfcc_transform = torchaudio.transforms.MFCC(
    sample_rate=48000, #36000 samples in 0.75s
    n_mfcc=40, # 40 MFCC features
    melkwargs={
        "n_fft": 2048,
        "hop_length": 512,
        "n_mels": 128
    }
)

data = MineData("./data/mine_impact_data_2019.mat", transform = mfcc_transform)

# Shuffle indices before splitting
all_indices = list(range(len(data)))
random.seed(42)  # for reproducibility
random.shuffle(all_indices)

# Split into train and test
train_id = all_indices[:3000]
test_id = all_indices[3000:]

# Subset the data
train_data = torch.utils.data.Subset(data, train_id)
test_data = torch.utils.data.Subset(data, test_id)

# DataLoaders
batch_size = 1

train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=True)

# Class mapping
class_map = {
    0: "drummy",
    1: "tight"
}

print(data[0][0].shape)

torch.Size([2840])


In [31]:
def init_train_var(model):
  criterion = nn.CrossEntropyLoss()
  optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)

  return criterion, optimizer

In [32]:
def train(train_loader, batch_size):
  device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

  model = models.Mine_MLP(nb_hidden=512)
  model.to(device)
  
  criterion, optimizer = init_train_var(model=model)
  history = {
        "train_loss": [],
        "train_acc": [],
        "val_loss": [],
        "val_acc": []
    }
  train_steps = len(train_loader.dataset) // batch_size
#   val_steps = len(val_loader.dataset) // batch_size

  nb_epochs = 10
  # run for nb_epochs
  for e in range(nb_epochs):
      # set the model in training mode
      model.train()
      # initialize the total training and validation loss
      epoch_train_loss = 0
    #   epoch_val_loss = 0
      # initialize the number of correct predictions in the training
      # and validation step
      train_correct = 0
    #   val_correct = 0

      for x, y in train_loader:
          
          x, y = x.to(device), y.to(device)
          optimizer.zero_grad()

          pred = model(x)
          loss = criterion(pred, y)
          loss.backward()
          optimizer.step()

          # add the loss to the total training loss so far and
          # calculate the number of correct predictions
          epoch_train_loss += loss
          train_correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    #   # switch off autograd for validation
    #   with torch.no_grad():
    #       # set the model in evaluation mode
    #       model.eval()
    #       # loop over the validation set
    #       for (x, y) in val_loader:

    #           x, y = x.to(device), y.to(device)
    #           pred = model(x)
    #           loss = criterion(pred, y)
    #           epoch_val_loss += loss
    #           val_correct += (pred.argmax(1) == y).type(torch.float).sum().item()



      # calculate the average epoch training and validation loss
      mean_train_loss = epoch_train_loss / train_steps
    #   mean_val_loss = epoch_val_loss / val_steps
      # calculate the training and validation accuracy
      train_correct = train_correct / len(train_loader.dataset)
      # val_correct = val_correct / len(val_loader.dataset)
      # update our training history
      history["train_loss"].append(mean_train_loss.cpu().detach().numpy())
      history["train_acc"].append(train_correct)
    #   history["val_loss"].append(mean_val_loss.cpu().detach().numpy())
      # history["val_acc"].append(val_correct)
      # print the model training and validation information
      print("[INFO] EPOCH: {}/{}".format(e + 1, nb_epochs))
      print("Train loss: {:.6f}, Train accuracy: {:.4f}".format(
          mean_train_loss, train_correct))
    #   print("Val loss: {:.6f}, Val accuracy: {:.4f}\n".format(
    #       mean_val_loss, val_correct))
      # save the model if the validation loss is less than the previous
      # if mean_val_loss - prev_mean_val_loss> 0.01:
      #   break
      # else:
      #   prev_mean_val_loss = mean_val_loss

  torch.save(model, "models_mine_mlp.pth")

In [33]:
def test(model_path, test_loader):
  # test on the test set
  print("[INFO] Testing the model")
  device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
  model = torch.load(model_path,weights_only=False)
  model.to(device)
  test_correct = 0
  with torch.no_grad():
      model.eval()
      for x, y in test_loader:
          x, y = x.to(device), y.to(device)
          pred = model(x)
          test_correct += (pred.argmax(1) == y).type(torch.float).sum().item()

  test_acc = test_correct / len(test_loader.dataset)
  print(f"Test accuracy: {test_acc:.4f}")

In [34]:
train(train_loader, batch_size=1)

[INFO] EPOCH: 1/10
Train loss: 2.353846, Train accuracy: 0.8073
[INFO] EPOCH: 2/10
Train loss: 0.655482, Train accuracy: 0.8693
[INFO] EPOCH: 3/10
Train loss: 0.300817, Train accuracy: 0.8960
[INFO] EPOCH: 4/10
Train loss: 0.203368, Train accuracy: 0.9220
[INFO] EPOCH: 5/10
Train loss: 0.156575, Train accuracy: 0.9327
[INFO] EPOCH: 6/10
Train loss: 0.148086, Train accuracy: 0.9407
[INFO] EPOCH: 7/10
Train loss: 0.110861, Train accuracy: 0.9563
[INFO] EPOCH: 8/10
Train loss: 0.083687, Train accuracy: 0.9663
[INFO] EPOCH: 9/10
Train loss: 0.091012, Train accuracy: 0.9680
[INFO] EPOCH: 10/10
Train loss: 0.066700, Train accuracy: 0.9767


In [35]:
test("models_mine_mlp.pth", test_loader)

[INFO] Testing the model
Test accuracy: 0.9417
